# Support Vector Machine (SVM) classification using Concrete-ML

In this tutorial, we will show how to create, train, and evaluate a Support Vector Machine (SVM) model using the Concrete-ML library, an open-source privacy-preserving machine learning framework based on fully homomorphic encryption (FHE).

This tutorial is cut in 2 parts:
1. A quick setup of a LinearSVC model with Concrete-ML
2. A more in-depth approach taking a closer look to the concrete-ml specifics


## Introduction

### Concrete-ML and useful links

> Concrete-ML is an open-source, privacy-preserving, machine learning inference framework based on fully homomorphic encryption (FHE). It enables data scientists without any prior knowledge of cryptography to automatically turn machine learning models into their FHE equivalent, using familiar APIs from Scikit-learn and PyTorch.
> 
> <cite>&mdash; [Zama documentation](https://docs.zama.ai/concrete-ml/)</cite>

This tutorial does not require extensive knowledge of Concrete-ML. Newcomers might nonetheless be interested in reading some of the introductory sections of the official documentation, such as:

- [What is Concrete-ML](https://docs.zama.ai/concrete-ml/)
- [Key Concepts](https://docs.zama.ai/concrete-ml/getting-started/concepts)

### Support Vector Machine

SVM is a machine learning algorithm for classification and regression. LinearSVC is an efficient implementation of SVM
that works best when the data is linearly separable. In this tutorial, we will use the [iris dataset](https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html) which is a common example for demonstrating how to work with SVMs.

Concrete-ML exposes a LinearSVC class which implements the
[scikit-learn LinearSVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html) interface, so you should feel right at home.

### Setup code

Just as in any machine learning project, let's start by importing some libraries and setting the dataset.

In [1]:
# display visualizations and plots in the notebook itself
%matplotlib inline

# import numpy and matplotlib
import numpy as np
import matplotlib.pyplot as plt

# import the iris dataset from sklearn, as well as some utilities and the LinearSVC for reference
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC as SklearnLinearSVC
from sklearn.metrics import accuracy_score

# import the concrete-ml LinearSVC implementation
from concrete.ml.sklearn.svm import LinearSVC as ConcreteLinearSVC

# Load the iris dataset and select the first 2 features 
iris = datasets.load_iris()

# Split the dataset into a training and a testing set
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.4, random_state=42)

## Part 1: Train a simple model with Concrete-ML

Let's first start by quickly scaffolding a Concrete-ML LinearSVC code, so we can see how easy and familiar it is.


In [5]:
# Train a model with scikit-learn LinearSVC, perform prediction and compute the accuracy
svm_sklearn = SklearnLinearSVC()
svm_sklearn.fit(X_train, y_train)
y_pred_sklearn = svm_sklearn.predict(X_test)
accuracy_sklearn = accuracy_score(y_test, y_pred_sklearn)

# Perform the same steps with the Concrete-ML LinearSVC implementation
svm_concrete = ConcreteLinearSVC()
svm_concrete.fit(X_train, y_train)
y_pred_concrete_clear = svm_concrete.predict(X_test)
accuracy_concrete_clear = accuracy_score(y_test, y_pred_concrete_clear)

# A circuit needs to be compiled to enable FHE execution
circuit = svm_concrete.compile(X_train)
circuit.client.keygen(force=False)
# Now that a circuit is compiled, the svm_concrete can predict value with FHE
y_pred_concrete_fhe = svm_concrete.predict(X_test, execute_in_fhe=True)
accuracy_concrete_fhe = accuracy_score(y_test, y_pred_concrete_fhe)

print(f"Scikit-learn Accuracy: {accuracy_sklearn:.4f}")
print(f"Concrete-ML Quantized Accuracy: {accuracy_concrete_clear:.4f}")
print(f"Concrete-ML FHE Accuracy: {accuracy_concrete_fhe:.4f}")

Scikit-learn Accuracy: 0.9833
Concrete-ML Quantized Accuracy: 0.6833
Concrete-ML FHE Accuracy: 0.6833


### Code explanation

Let's have a more in-depth look at the code.

#### First, we have a regular scikit-learn LinearSVC example.

```python
# Load the iris dataset and select the first 2 features 
iris = datasets.load_iris()

# Split the dataset into a training and a testing set
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.4, random_state=42)

# Train a model with scikit-learn LinearSVC, perform prediction and compute the accuracy
svm_sklearn = SklearnLinearSVC()
svm_sklearn.fit(X_train, y_train)
y_pred_sklearn = svm_sklearn.predict(X_test)
accuracy_sklearn = accuracy_score(y_test, y_pred_sklearn)
```

Hopefully should not be confused by this, otherwise you may want to read the [official scikit-learn SVM documentation](https://scikit-learn.org/stable/modules/svm.html#svm-classification). 

The algorithm can be tweaked with the parameters exposed by the LinearSVC class, refer to the  [LinearSVC API documentation](https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html) as to what can be customized.

#### Second, we have a Concrete-ML implementation,which behaves just like the scikit-learn one.

```python
# Perform the same steps with the Concrete-ML LinearSVC implementation
svm_concrete = ConcreteLinearSVC()
svm_concrete.fit(X_train, y_train)
y_pred_concrete_clear = svm_concrete.predict(X_test)
accuracy_concrete_clear = accuracy_score(y_test, y_pred_concrete_clear)
```

One thing to note here is not only the model is trained using clear data, but the prediction are also performed in a *plain environment*, there is no encryption at this stage.

In order to perform prediction in a FHE environment, the model first has to be compiled into a circuit.

#### Third, the model is compiled to enable FHE execution

```python
# A circuit needs to be compiled to enable FHE execution
circuit = svm_concrete.compile(X_train)
circuit.client.keygen(force=False)
# Now that a circuit is compiled, the svm_concrete can predict value with FHE
y_pred_concrete_fhe = svm_concrete.predict(X_test, execute_in_fhe=True)
accuracy_concrete_fhe = accuracy_score(y_test, y_pred_concrete_fhe)
```

Now that the model is compiled, computing predictions with FHE is just a matter of passing an `execution_in_fhe` parameter set to `True`.


#### Accuracy

Finally, we can measure the accuracy of our 3 different predictions:

```python
print(f"Scikit-learn Accuracy: {accuracy_sklearn:.4f}")
print(f"Concrete-ML Clear Accuracy: {accuracy_concrete_clear:.4f}")
print(f"Concrete-ML FHE Accuracy: {accuracy_concrete_fhe:.4f}")
```

### Key takeaways

#### Simplicity of execution

For a high-level use-case, Concrete-ML offers a very similar interface to scikit-learn. The main difference is *a model needs to be compiled to allow execution in FHE*.

#### Model Accuracy

Concrete-ML prediction accuracy is slightly under scikit-learn implementation. This is because of [quantization](https://docs.zama.ai/concrete-ml/advanced-topics/quantization): number precision needs to be fixed-size for the model to be evaluated in FHE. This can be alleviated down to where the accuracy difference is none or negligible.

#### Execution time

The execution is slower with Concrete-ML, especially when compiling the model. Enabling and using encryption indeed uses a lot more resources than training and using a model using plain data. The speed can be increased by *reducing the precision of the data* (understand diminish the fixed-size number precision). Depending on the project, you thus have to choose between:

- a slower model that performs accurate predictions
- a faster model that performs less accurate predictions

## Part 2: In-Depth model development

A more in-depth approach, showing how to effectively develop with concrete-ml. This will quote and follow the steps of [model development](https://docs.zama.ai/concrete-ml/getting-started/concepts#i.-model-development)

Especially:
- the effects of quantization and finding the good bit number
- setup the virtual library to speed up the development workflow
- use inference to use the model with encrypted data

---

### Step a: training the model

Nothing new under the sun here, we need to train a relevant model for our machine-learning problem. As we relied previously on the Iris example and improving it is outside the scope of this tutorial, we can just take back what we used so far.

In [ ]:
# setup and train a scikit-learn LinearSVC model, just as before
svm_sklearn = SklearnLinearSVC()
svm_sklearn.fit(X_train, y_train)
# predict some test data and measure the model accuracy
y_pred_sklearn = svm_sklearn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_sklearn)

print(f"Scikit-learn Accuracy: {accuracy_sklearn:.4f}")

Not too shabby.

### Step b: quantize the model

So far we conveniently avoided most of Concrete-ML specificities for the sake of simplicity. The first Concrete-ML specific step of developping a model is to quantize it, which soberly means to turn the model into an integer equivalent.

Although you are strongly encouraged to read the [Zama introduction to quantization](https://docs.zama.ai/concrete-ml/advanced-topics/quantization), the key takeaway is **a model needs to be reduced to a *discrete*, smaller set in order for the encryption to happen**. Otherwise the data becomes too large to be manipulated in FHE. 

As of v0.6.0 the maximum bit size is 8. The lighter the bit size the more efficient the concrete-ml model is. Thus the goal of the quantization step is to find the lowest bit size value that offers an acceptable accuracy, so the model efficiency is maximized.

In [ ]:
# compute the accuracy of a n_bit quantized linear ranging from 2 to 8 bits
for n_bits in range(2, 9):
    svm_concrete = ConcreteLinearSVC(n_bits)
    svm_concrete.fit(X_train, y_train)
    y_pred = svm_concrete.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{n_bits} Bits Quantized Accuracy: {accuracy:.4f}")

### Step c: simulate the model execution

Executing models with FHE can prove to be a slow process, depending on:
- the data-set size
- the model itself
- the hardware executing the model

Concrete-ML allows to simulate FHE model execution using the [virtual library](https://docs.zama.ai/concrete-ml/advanced-topics/compilation#simulation-with-the-virtual-library). This speeds up the development process considerably by avoiding long-running compilation to be repeated every time.

> Testing FHE models on very large data-sets can take a long time. Furthermore, not all models are compatible with FHE constraints out-of-the-box. Simulation using the Virtual Library allows you to execute a model that was quantized, to measure the accuracy it would have in FHE, but also to determine the modifications required to make it FHE compatible.
>
> — [Zama documentation](https://docs.zama.ai/concrete-ml/getting-started/concepts#i.-model-development)


In [ ]:
# import the configuration from concrete-numpy
from concrete.numpy import Configuration

# define a configuration
COMPIL_CONFIG_VL = Configuration(
    dump_artifacts_on_unexpected_failures=False,
    enable_unsafe_features=True,
)

# arbitrarily set the bit size to 8
n_bits = 8
svm_concrete = ConcreteLinearSVC(n_bits)
svm_concrete.fit(X_train, y_train)

# compile the model with virtual lib enabled and with the defined configuration
circuit = svm_concrete.compile(X_train, use_virtual_lib=True, configuration=COMPIL_CONFIG_VL)

# the model can now be executed with FHE
y_pred = svm_concrete.predict(X_test, execute_in_fhe=True)
accuracy = accuracy_score(y_test, y_pred)
print(f"{n_bits} Bits FHE Accuracy: {accuracy:.4f}")

*The virtual library enables some unsafe features, so it should understandably not be used for production*

So far so good, the model is compiled and executed much quicker with the virtual library, allowing us to run it with different configurations.

In a more complex scenario, we would want to fine-tune a lot of model parameters, however here for the sake of simplicity we kept the model to its default configuration and are only left to play with the bit size.

We can now put the two previous parts together and make sure our quantized model prediction stay accurate in FHE

In [ ]:
for n_bits in range(2, 9):
    svm_concrete = ConcreteLinearSVC(n_bits)
    svm_concrete.fit(X_train, y_train)
    svm_concrete.compile(X_train, use_virtual_lib=True, configuration=COMPIL_CONFIG_VL) # the model is now compiled
    y_pred = svm_concrete.predict(X_test, execute_in_fhe=True) # the execution is done in FHE
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{n_bits} Bits Quantized Accuracy: {accuracy:.4f}")

The model predictions in FHE (with virtual library) are aligned with the predictions of the plain, quantized model.

We now need to settle for a bit size. Depending on the model use case we might want to favor speed or accuracy. We can consider 2 different use-case of our iris recognition model. One is a "machine-learning as a service" model, in which users would request our model to analyze their data. In such a context we might want to favorise speed (thus execution time and reduce computation cost) and select a smaller bitsize, such as 4. On the contrary we can also invision our model as being used by scientists to classify irises, where computation costs are not as much of an issue, but requires the best possible accuracy. This scenario would lead us to select a bitsize to 6, as it is the lowest bitsize that provides the best accuracy (0.9833).

In both scenarios, by putting a little more time in testing and selecting our bitsize, we managed to reduce it from 25% to 50%, avoiding unnecessary computation efforts and speeding up the production model.

### Step d: compile the model

Now that we have selected a relevant bit-size we can compile the model, without virtual lib, so we can use it in production

In [ ]:
# set up and train a 6bit quantized LinearSVC model
svm_concrete = ConcreteLinearSVC(4)
svm_concrete.fit(X_train, y_train)

# compile the model and generate a key
circuit = svm_concrete.compile(X_train)
circuit.client.keygen(force=False)

# predict the test set to verify the compiled model accuracy
y_pred = svm_concrete.predict(X_test, execute_in_fhe=True)
accuracy = accuracy_score(y_test, y_pred)

## Conclusion

Setting up FHE with Concrete-ML on a LinearSVC model is very simple, in the regard that Concrete-ML provides an implementation of the [scikit-learn LinearSVC interface](https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html). As a matter of fact, a working FHE model can be setup with just a few lines of code.

Setting up a model with FHE benefits nonetheless from some additional work. For LinearSVC models, the main point is to select a relevant bit-size for [quantizing](https://docs.zama.ai/concrete-ml/advanced-topics/quantization) the model. Some additional tools can smooth up the development workflow, such as alleviating the [compilation](https://docs.zama.ai/concrete-ml/advanced-topics/compilation) time by making use of the [virtual library](https://docs.zama.ai/concrete-ml/advanced-topics/compilation#simulation-with-the-virtual-library) 

Once the model is carefully trained and quantized, it is ready to be deployed and used in production. Here are some useful links that cover this subject:
- [Inference in the Cloud](https://docs.zama.ai/concrete-ml/getting-started/cloud) summarize the steps for cloud deployment
- [Production Deployment](https://docs.zama.ai/concrete-ml/advanced-topics/client_server) offers a high-level view of how to deploy a Concrete-ML model in a client/server setting.
- [Client Server in Concrete ML](https://github.com/zama-ai/concrete-ml/blob/release/0.6.x/docs/advanced_examples/ClientServer.ipynb) provides a more hands-on approach as another tutorial.